In [35]:
from numpy.random import multinomial
import numpy as np

In [36]:
N = 10 
C = 3

current_pop = np.random.randint(0, 100, size=N)
prob = np.zeros((C, N), dtype=np.float64)
print(current_pop)

[92 80 62 51 57 68 22 40 15 33]


In [37]:
prob[1] = 1 - np.sum(prob, axis=0)

In [38]:
multinomial(current_pop[0], prob.T[0])

array([ 0, 92,  0])

In [50]:
import numpy as np

def chain_multinomial(n, p, stay_idx, rng=None, check_tol=1e-12):
    """
    Competing-risks multinomial with an explicit 'stay' category.

    Args:
        n (int): Number of individuals in the source compartment.
        p (array-like): Length-K vector. For k != stay_idx, p[k] = r_k * dt.
        stay_idx (int): Index of the 'stay' category.
        rng : np.random.Generator or seed or None. If None, uses default_rng().

    Returns:
        counts (np.ndarray of int, shape (K,)): Realized counts for each category (including 'stay'), summing to n.
    """
    p = np.asarray(p, dtype=float)
    K = p.size # number of categories
    if not (0 <= stay_idx < K):
        raise IndexError("stay_idx out of bounds.")
    if np.any(p < -1e-16):
        raise ValueError("All entries in p must be non-negative.")

    # Aggregate total hazard H = sum of non-stay components
    dest_mask = np.ones(K, dtype=bool)
    dest_mask[stay_idx] = False
    h = p[dest_mask] 
    H = float(h.sum())

    rng = np.random.default_rng(rng)
    counts = np.zeros(K, dtype=int)

    if H <= check_tol:
        # No hazard -> everyone stays
        counts[stay_idx] = n
        return counts

    # Probability of leaving given total hazard H
    p_leave = -np.expm1(-H)
    p_leave = min(max(p_leave, 0.0), 1.0)  # clamp for FP safety

    # Draw total exits
    exits = rng.binomial(n, p_leave)
    counts[stay_idx] = n - exits

    if exits == 0:
        return counts

    # Conditional probabilities across destinations q_k = h_k / H
    q = h / H
    # Allocate exits across destinations
    alloc = rng.multinomial(exits, q)

    # Write back
    counts[np.where(dest_mask)[0]] = alloc
    return counts


In [61]:

%timeit chain_multinomial(n, p, stay_idx)



84.3 µs ± 7.38 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [63]:
%timeit multinomial(n, p)

1.43 µs ± 15.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [48]:
-np.expm1(-0.9)

0.5934303402594009

In [49]:
1 - np.exp(-0.9)

0.5934303402594009